In [1]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, text, inspect, func

# data
import pandas as pd
import numpy as np

# viz
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
SQL_USERNAME = "postgres"
SQL_PASSWORD = "postgres" # change this
SQL_IP = "localhost"
PORT = 5432
DATABASE = "crowdfunding" # change this

In [3]:
connection_string = f"postgresql+psycopg2://{SQL_USERNAME}:{SQL_PASSWORD}@{SQL_IP}:{PORT}/{DATABASE}"
engine = create_engine(connection_string)

In [4]:
# INSPECT

# Create the inspector and connect it to the engine
inspector_gadget = inspect(engine)

# Collect the names of tables within the database
tables = inspector_gadget.get_table_names()

# print metadata for each table
for table in tables:
    print(table)
    print("-----------")
    
    # get columns
    columns = inspector_gadget.get_columns(table)
    for column in columns:
        print(column["name"], column["type"])

    print()

contacts_df
-----------
contact_id INTEGER
first_name VARCHAR(250)
last_name VARCHAR(250)
email VARCHAR(250)

campaign_df
-----------
cf_id INTEGER
contact_id INTEGER
company_name VARCHAR(250)
description VARCHAR(250)
goal DOUBLE PRECISION
pledged DOUBLE PRECISION
outcome VARCHAR(250)
backers_count INTEGER
country VARCHAR(250)
currency VARCHAR(10)
launch_date TIMESTAMP
end_date TIMESTAMP
category_id VARCHAR(50)
subcategory_id VARCHAR(250)

category_df
-----------
category_id VARCHAR(50)
category VARCHAR(250)

subcategory_df
-----------
subcategory_id VARCHAR(50)
subcategory VARCHAR(250)



In [5]:
# Declare a Base using `automap_base()`
Base = automap_base()

# Use the Base class to reflect the database tables
Base.prepare(autoload_with=engine)

# Print all of the classes mapped to the Base
print(list(Base.classes))

[<class 'sqlalchemy.ext.automap.subcategory_df'>, <class 'sqlalchemy.ext.automap.category_df'>, <class 'sqlalchemy.ext.automap.contacts_df'>, <class 'sqlalchemy.ext.automap.campaign_df'>]


In [57]:
query = """
SELECT category_id, subcategory_id, outcome, COUNT(outcome)
FROM campaign_df
GROUP BY category_id, subcategory_id, outcome;
  ;
;"""

outcome_df = pd.read_sql(text(query), con=engine)
outcome_df.head(10)

,category_id,subcategory_id,outcome,count
0,cat5,subcat11,live,2
1,cat6,subcat14,failed,7
2,cat6,subcat14,successful,9
3,cat5,subcat13,successful,9
4,cat6,subcat10,successful,13
5,cat5,subcat20,successful,11
6,cat5,subcat5,successful,34
7,cat3,subcat9,failed,16
8,cat6,subcat16,successful,4
9,cat5,subcat20,failed,3


In [59]:
import plotly.express as px

In [65]:
fig=px.treemap(outcome_df, path=['outcome','category_id','subcategory_id'], values='count',color='count')
fig.update_layout(
    margin=dict(l=10,r=10,t=50,b=10),
    autosize=False,
    height=800,
    width=1500,
    title=("Outcome grouped by category and subcategory")
)
fig.show()